In [1]:
import sys
import glob
sys.path.insert(0, '../libs')

import process_files
import pandas as pd
from datetime import datetime

import time

import os

In [2]:
variable_files_path = r'../data/variables/var-mes-*-fill.zip'
#estation_completitud_path = r'../data/variables_profiles/CUM70_todas.csv'
variable_full_output = r'../data/variables/var-mes-todas-fill-union.csv'
variable_full_tidy_output = r'../data/variables/var-mes-todas-fill-union-col.csv'

In [3]:
_path = variable_files_path.split('/')
_files = glob.glob(os.path.join(*_path))
print(_files)

['..\\data\\variables\\var-mes-PTPM_CON-fill.zip', '..\\data\\variables\\var-mes-Q_MEDIA_D-fill.zip', '..\\data\\variables\\var-mes-Q_MN_D-fill.zip', '..\\data\\variables\\var-mes-Q_MX_D-fill.zip', '..\\data\\variables\\var-mes-TMN_CON-fill.zip', '..\\data\\variables\\var-mes-TMX_CON-fill.zip']


In [4]:
dtypes = { 'CodigoEstacion':'string', 'Etiqueta':'string'}
usecols = ['CodigoEstacion', 'Etiqueta', 'Fecha', 'min', 'median', 'max', 'count']
#dtypesEstacionesCompletas = { 'codigo':'string'}
#usecolsEstacionesCompletas = ['codigo']
etiqueta_valor = {'PTPM_CON':['min','median','max', 'count'], 
                  'Q_MN_D':['min', 'count'],
                  'Q_MEDIA_D':['min','median','max','count'],
                  'Q_MX_D':['max', 'count'],
                  'TMN_CON':['min', 'count'],
                  'TMX_CON':['max', 'count']
                  }
dateparse = lambda x: datetime.strptime(x, '%Y-%m')

In [5]:
start_time = time.time()

#print(f'..... h:{time.asctime()} f:{estation_completitud_path} leyendo completas .....')
#dfcom = pd.read_csv(estation_completitud_path, dtype = dtypesEstacionesCompletas)
#print(f'----- r:{dfcom.shape} h:{time.asctime()} t:{(time.time() - start_time) / 60} -----')

In [6]:
def unificar(f):
    print(f'##### h:{time.asctime()} f:{f} leyendo #####')
    df = pd.read_csv(f, dtype = dtypes, usecols = usecols)
    etiqueta = df.iat[0,1]
    print(f'----- h:{time.asctime()} t:{(time.time() - start_time) / 60} {etiqueta} {df.shape}-----')
    
    #print(f'..... h:{time.asctime()} e:{etiqueta} Filtrando completas .....')
    #df = df.merge(dfcom, left_on='CodigoEstacion', right_on='codigo', how = 'inner' )
    #df.drop('codigo', axis = 'columns', inplace = True)
    df.columns = ['codigo', 'etiqueta', 'fecha', 'min', 'median', 'max', 'count']
    #print(f'----- h:{time.asctime()} t:{(time.time() - start_time) / 60} {df.shape} -----')
    
    dfs = []
    
    for valor in etiqueta_valor[etiqueta]:
        print(f'..... h:{time.asctime()} v:{valor} creando df .....')
        dfc = df[['codigo', 'etiqueta', 'fecha', valor]].copy()
        dfc['etiqueta'] = dfc['etiqueta'] + '_' + valor
        dfc.columns = ['codigo', 'etiqueta', 'fecha', 'valor']
        dfs.append(dfc)
        print(f'----- h:{time.asctime()} t:{(time.time() - start_time) / 60} {dfc.shape} -----')
    
    dfs = pd.concat(dfs, axis = 0)
    print(f'***** h:{time.asctime()} t:{(time.time() - start_time) / 60} {dfs.shape} *****')
    
    return dfs
    #return df

In [7]:
%%time
res = process_files.processFiles(variable_files_path, unificar)

['..\\data\\variables\\var-mes-PTPM_CON-fill.zip', '..\\data\\variables\\var-mes-Q_MEDIA_D-fill.zip', '..\\data\\variables\\var-mes-Q_MN_D-fill.zip', '..\\data\\variables\\var-mes-Q_MX_D-fill.zip', '..\\data\\variables\\var-mes-TMN_CON-fill.zip', '..\\data\\variables\\var-mes-TMX_CON-fill.zip']
##### h:Sun Jul 26 18:39:53 2020 f:..\data\variables\var-mes-PTPM_CON-fill.zip leyendo #####
----- h:Sun Jul 26 18:39:57 2020 t:0.11188269853591919 PTPM_CON (614220, 7)-----
..... h:Sun Jul 26 18:39:57 2020 v:min creando df .....
----- h:Sun Jul 26 18:39:58 2020 t:0.11776609818140665 (614220, 4) -----
..... h:Sun Jul 26 18:39:58 2020 v:median creando df .....
----- h:Sun Jul 26 18:39:58 2020 t:0.12388273477554321 (614220, 4) -----
..... h:Sun Jul 26 18:39:58 2020 v:max creando df .....
----- h:Sun Jul 26 18:39:58 2020 t:0.12943272988001506 (614220, 4) -----
..... h:Sun Jul 26 18:39:58 2020 v:count creando df .....
----- h:Sun Jul 26 18:39:59 2020 t:0.1348660429318746 (614220, 4) -----
***** h:Su

In [8]:
%%time
dfs = pd.concat([r['result'] for r in res])

Wall time: 394 ms


In [9]:
%%time
#Almacenar una variable por fila
dfs.to_csv(variable_full_output, index = False)

Wall time: 20.9 s


In [10]:
%%time
dfs.set_index(['codigo', 'etiqueta', 'fecha'], inplace=True)

Wall time: 2.95 s


In [11]:
%%time
#pasar las variables a las columnas
dfs = dfs.unstack('etiqueta').reset_index()
dfs[dfs.filter(regex='count').columns] = dfs[dfs.filter(regex='count').columns].fillna(0)

Wall time: 3.65 s


In [12]:
%%time
dfs.to_csv(variable_full_tidy_output, index = False)

Wall time: 28.2 s
